In [200]:
import pydeck as pdk
import pandas as pd
import json

In [201]:
df = pd.read_csv("final.csv")
df = df.fillna(0)
print(df)

       exporter        importer  1970  1971  1972  1973  1974  1975  1976  \
0      Albania,    Burkina Faso   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
1      Albania,               0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
2      Albania,           Total   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
3      Algeria,          Angola   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
4      Algeria,  Western Sahara   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
...         ...             ...   ...   ...   ...   ...   ...   ...   ...   
3036    Zambia,               0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
3037    Zambia,           Total   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
3038  Zimbabwe,         Czechia   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
3039  Zimbabwe,               0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
3040  Zimbabwe,           Total   0.0   0.0   0.0   0.0   0.0   0.0   0.0   

      1977  ...  2012  2013  2014  2015  2016  2017  2018  2019  2020  Tota

In [202]:
with open("./country-capitals.json") as f:
    countries = json.load(f)
    
print(countries[0])

{'CountryName': 'Somaliland', 'CapitalName': 'Hargeisa', 'CapitalLatitude': '9.55', 'CapitalLongitude': '44.050000', 'CountryCode': 'NULL', 'ContinentName': 'Africa'}


In [203]:
data = {}
for country in countries:
    data[country["CountryName"]] = (country["CapitalLatitude"], country["CapitalLongitude"])
    
print(data["United States"])

country_set = set.union(set(df['exporter'].tolist()), set(df['importer'].tolist()))
print(len(country_set))

locations = {val: data[val] for val in data if (val in country_set or (val + "," in country_set))}
print(len(locations))

('38.883333', '-77.000000')
378
177


In [204]:
export_long = []
export_lat = []

for location in df["exporter"].tolist():
    location = location.replace(",", "")
    if location in locations:
        export_long.append(float(locations[location][0]))
        export_lat.append(float(locations[location][1]))
    else:
        export_long.append(np.nan)
        export_lat.append(np.nan)

df["lng_h"] = export_long
df["lat_h"] = export_lat

In [205]:
import_long = []
import_lat = []

for location in df["importer"].tolist():
    if location in locations:
        import_long.append(float(locations[location][0]))
        import_lat.append(float(locations[location][1]))
    else:
        import_long.append(np.nan)
        import_lat.append(np.nan)

df["lng_w"] = import_long
df["lat_w"] = import_lat

In [206]:
df = df[df['lng_w'].notna()]
df = df[df['lat_w'].notna()]

df = df[df['lng_h'].notna()]
df = df[df['lat_h'].notna()]

In [207]:
df.to_csv("data.csv")
df1 = df.head(20)

In [208]:
df = df.reset_index()
print(df)

      index    exporter        importer  1970  1971  1972  1973  1974  1975  \
0         0    Albania,    Burkina Faso   0.0   0.0   0.0   0.0   0.0   0.0   
1         3    Algeria,          Angola   0.0   0.0   0.0   0.0   0.0   0.0   
2         4    Algeria,  Western Sahara   0.0   0.0   0.0   0.0   0.0   0.0   
3         8     Angola,   Cote d'Ivoire   0.0   0.0   0.0   0.0   0.0   0.0   
4         9     Angola,   Guinea-Bissau   0.0   0.0   0.0   0.0   0.0   0.0   
...     ...         ...             ...   ...   ...   ...   ...   ...   ...   
1810   2883  Venezuela,         Bolivia   0.0   0.0   0.0   0.0   0.0  13.0   
1811   2884  Venezuela,         Ecuador   0.0   0.0   0.0   0.0   0.0   0.0   
1812   2885  Venezuela,       Guatemala   0.0   0.0   0.0   0.0   0.0   0.0   
1813   2886  Venezuela,        Suriname   0.0   0.0   0.0   0.0   0.0   0.0   
1814   3035     Zambia,         Liberia   0.0   0.0   0.0   0.0   0.0   0.0   

      1976  ...  2016  2017  2018  2019  2020  Tota

In [209]:
GREEN_RGB = [0, 255, 0, 40]
RED_RGB = [240, 100, 0, 40]

max_val = max(df["Total"].tolist())
print(max_val)

# Specify a deck.gl ArcLayer
arc_layer = pdk.Layer(
    "ArcLayer",
    data=df,
    get_width="Total/1000",
    get_source_position=["lat_h", "lng_h"],
    get_target_position=["lat_w", "lng_w"],
    get_tilt=0,
    get_source_color=RED_RGB,
    get_target_color=GREEN_RGB,
    pickable=True,
    auto_highlight=True,
)

view_state = pdk.ViewState(latitude=37.7576171, longitude=-122.5776844, bearing=0, pitch=0, zoom=1)


TOOLTIP_TEXT = {"html": "{Total} Arms Trade from 1970 - Present"}
r = pdk.Deck(arc_layer, initial_view_state=view_state, tooltip=TOOLTIP_TEXT)
r.to_html("arc_layer.html")

41676.0


In [ ]:
import pydeck as pdk
import pandas as pd

DATA_URL = "https://raw.githubusercontent.com/ajduberstein/sf_public_data/master/bay_area_commute_routes.csv"
# A bounding box for downtown San Francisco, to help filter this commuter data
DOWNTOWN_BOUNDING_BOX = [
    -122.43135291617365,
    37.766492914983864,
    -122.38706428091974,
    37.80583561830737,
]


def in_bounding_box(point):
    """Determine whether a point is in our downtown bounding box"""
    lng, lat = point
    in_lng_bounds = DOWNTOWN_BOUNDING_BOX[0] <= lng <= DOWNTOWN_BOUNDING_BOX[2]
    in_lat_bounds = DOWNTOWN_BOUNDING_BOX[1] <= lat <= DOWNTOWN_BOUNDING_BOX[3]
    return in_lng_bounds and in_lat_bounds


df = pd.read_csv(DATA_URL)
# Filter to bounding box
df = df[df[["lng_w", "lat_w"]].apply(lambda row: in_bounding_box(row), axis=1)]

df.to_csv("test.csv")

GREEN_RGB = [0, 255, 0, 40]
RED_RGB = [240, 100, 0, 40]

# Specify a deck.gl ArcLayer
arc_layer = pdk.Layer(
    "ArcLayer",
    data=df,
    get_width="2",
    get_source_position=["0.0", "0.0"],
    get_target_position=["1.0", "1.0"],
    get_tilt=15,
    get_source_color=RED_RGB,
    get_target_color=GREEN_RGB,
    pickable=True,
    auto_highlight=True,
)

view_state = pdk.ViewState(latitude=37.7576171, longitude=-122.5776844, bearing=45, pitch=50, zoom=8,)


TOOLTIP_TEXT = {"html": "{S000} jobs <br /> Home of commuter in red; work location in green"}
r = pdk.Deck(arc_layer, initial_view_state=view_state, tooltip=TOOLTIP_TEXT)
r.to_html("arc_layer.html")